In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [3]:
toron=pd.read_excel('toronto.xlsx')

In [4]:
toron.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
toron=toron[toron.Borough != 'Not assigned']

Aparently the merging has been already done

In [6]:
toron['Postal Code']=='M5A'

2      False
3      False
4       True
5      False
6      False
8      False
9      False
11     False
12     False
13     False
14     False
17     False
18     False
20     False
21     False
22     False
23     False
26     False
27     False
30     False
31     False
32     False
36     False
39     False
40     False
41     False
45     False
46     False
47     False
48     False
49     False
50     False
54     False
55     False
56     False
57     False
58     False
59     False
63     False
64     False
65     False
66     False
67     False
68     False
72     False
73     False
74     False
75     False
76     False
77     False
80     False
81     False
82     False
83     False
84     False
85     False
86     False
89     False
90     False
91     False
92     False
93     False
94     False
95     False
98     False
99     False
100    False
102    False
103    False
104    False
107    False
108    False
109    False
111    False
112    False
113    False
114    False

In [7]:
toron.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
toron = toron.reset_index(drop=True)

In [9]:
toron.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Neighborhood column has no not assigned values

In [10]:
toron['Neighborhood']=='Not assigned'

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
30     False
31     False
32     False
33     False
34     False
35     False
36     False
37     False
38     False
39     False
40     False
41     False
42     False
43     False
44     False
45     False
46     False
47     False
48     False
49     False
50     False
51     False
52     False
53     False
54     False
55     False
56     False
57     False
58     False
59     False
60     False
61     False
62     False
63     False
64     False
65     False
66     False
67     False
68     False
69     False
70     False
71     False
72     False
73     False
74     False
75     False
76     False

In [11]:
toron.shape

(103, 3)

In [17]:
pip install geocoder

     |████████████████████████████████| 102kB 6.1MB/s ta 0:00:011
     |████████████████████████████████| 92kB 16.3MB/s eta 0:00:01
     |████████████████████████████████| 829kB 11.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install geopandas

     |████████████████████████████████| 931kB 7.3MB/s eta 0:00:01███████████████   | 839kB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 34.5MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 5.4MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 5.2MB/s eta 0:00:01                    | 1.6MB 5.2MB/s eta 0:00:02
  Found existing installation: pyproj 1.9.6
    Uninstalling pyproj-1.9.6:
      Successfully uninstalled pyproj-1.9.6
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [12]:
toron['ADDRESS']= toron['Postal Code'] + ', Toronto, Ontario'

In [13]:
toron.head()

,Postal Code,Borough,Neighborhood,ADDRESS
0,M3A,North York,Parkwoods,"M3A, Toronto, Ontario"
1,M4A,North York,Victoria Village,"M4A, Toronto, Ontario"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","M5A, Toronto, Ontario"
3,M6A,North York,"Lawrence Manor, Lawrence Heights","M6A, Toronto, Ontario"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","M7A, Toronto, Ontario"


Applied a code to find latitude and longitude from geopy

In [42]:
from geopy.extra.rate_limiter import RateLimiter
locator = Nominatim(user_agent= 'myGeocoder')
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
toron['location'] = toron['ADDRESS'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
toron['point'] = toron['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
toron[['latitude', 'longitude', 'altitude']] = pd.DataFrame(toron['point'].tolist(), index=toron.index)


In [43]:
toron.head(25)

,Postal Code,Borough,Neighborhood,ADDRESS,location,point,latitude,longitude,altitude
0,M3A,North York,Parkwoods,"M3A, Toronto, Ontario","(Toronto, Golden Horseshoe, Ontario, M5H 2N2, ...","(43.6534817, -79.3839347, 0.0)",43.653482,-79.383935,0.0
1,M4A,North York,Victoria Village,"M4A, Toronto, Ontario",None,None,NaN,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","M5A, Toronto, Ontario",None,None,NaN,NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights","M6A, Toronto, Ontario",None,None,NaN,NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","M7A, Toronto, Ontario","(Toronto, Golden Horseshoe, Ontario, M5H 2N2, ...","(43.6534817, -79.3839347, 0.0)",43.653482,-79.383935,0.0
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village","M9A, Toronto, Ontario",None,None,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge","M1B, Toronto, Ontario","(Toronto, Golden Horseshoe, Ontario, M5H 2N2, ...","(43.6534817, -79.3839347, 0.0)",43.653482,-79.383935,0.0
7,M3B,North York,Don Mills,"M3B, Toronto, Ontario",None,None,NaN,NaN,NaN
8,M4B,East York,"Parkview Hill, Woodbine Gardens","M4B, Toronto, Ontario",None,None,NaN,NaN,NaN
9,M5B,Downtown Toronto,"Garden District, Ryerson","M5B, Toronto, Ontario",None,None,NaN,NaN,NaN


The code worked but there are some results that were not found so I proceeded to use the given package

In [14]:
geo=pd.read_csv('Geospatial_Coordinates.csv')

In [15]:
toron2=pd.merge(toron, geo, on='Postal Code')
toron2.head()            
    

,Postal Code,Borough,Neighborhood,ADDRESS,Latitude,Longitude
0,M3A,North York,Parkwoods,"M3A, Toronto, Ontario",43.753259,-79.329656
1,M4A,North York,Victoria Village,"M4A, Toronto, Ontario",43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","M5A, Toronto, Ontario",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights","M6A, Toronto, Ontario",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","M7A, Toronto, Ontario",43.662301,-79.389494


In [16]:
toron2 =toron2.drop(['ADDRESS'], axis=1 )

In [17]:
toron2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
